In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import openai
from scipy.spatial.distance import cosine

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel('ds1 sample.xlsx')

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for each row in the dataset
data_strings = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)
embeddings = model.encode(data_strings, convert_to_tensor=True)

# Save the embeddings for future reference (optional)
torch.save(embeddings, 'data_embeddings.pt')

# Set up OpenAI API key
openai.api_key = 'Use your OpenAI API key'  # Replace with your OpenAI API key

# Improved function to create a more structured prompt
def create_structured_prompt(question, context):
    prompt = (
        f"Data Context: {context}\n"
        f"Question: {question}\n"
        "Instructions: Analyze the data context above and provide a detailed answer to the question."
    )
    return prompt

# Improved function to find the most relevant row(s) based on a question
def find_most_relevant_rows(question, model, embeddings, df):
    # Embed the user's question
    question_embedding = model.encode(question, convert_to_tensor=True).cpu()

    # Ensure that all embeddings are on the CPU
    embeddings = [embedding.cpu() for embedding in embeddings]

    # Calculate cosine similarities
    similarities = [1 - cosine(question_embedding.numpy(), row_embedding.numpy()) for row_embedding in embeddings]

    # Get the top N most similar rows (N = 1 for simplicity)
    top_row_idx = torch.argmax(torch.tensor(similarities)).item()

    # Return the most relevant row as context
    return df.iloc[top_row_idx]

# Function to query OpenAI with the relevant context and the user's question
def query_openai_with_context(question, context):
    prompt = create_structured_prompt(question, context)
    
    # Query OpenAI using the correct API for chat-based models
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use "gpt-4" if you have access to it
        messages=[
            {"role": "system", "content": "You are a data analyst specializing in educational data."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300  # Adjust this as needed
    )
    
    return response['choices'][0]['message']['content'].strip()

# Example: A loop to handle multiple user questions
while True:
    user_question = input("Ask a question about the data (or type 'exit' to quit): ")
    if user_question.lower() == 'exit':
        break
    
    relevant_row = find_most_relevant_rows(user_question, model, embeddings, df)
    response = query_openai_with_context(user_question, relevant_row)
    print("Answer:", response)


/Users/harishanand/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/harishanand/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Answer: To determine which academic year had the highest average GPA before COVID-19, we need to look at the "Level/Year" and "Your cumulative average (GPA)" responses in the data context provided.

In this case, the "Level/Year" response is "First/Freshman" and the "Your cumulative average (GPA)" response is "80-89 / 3-3.49". From this data, we can infer that the academic year with the highest average GPA before COVID-19 was the first year of the student's undergraduate study, which corresponds to the freshman year.

Therefore, the academic year that had the highest average GPA before COVID-19 was the freshman year.

If you have any more questions or need further analysis, feel free to ask!
Answer: Based on the data context provided, the most commonly used digital tool by students before COVID-19 was a laptop. This information is indicated in the responses under the question: "Before COVID-19: Which of the following digital tools do you usually use?" where the student answered "Laptop